#### Link
- https://jadhav-pritish.medium.com/simple-tricks-to-speed-up-pandas-by-100x-3b7e705783a8

### Load data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(r"..\_datasets\titanic.csv.gz")
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df = data.copy()

In [30]:
%%timeit
for x in df['PassengerId']:
    filtro = df["PassengerId"]==x
    temp = df[filtro]
    NewCabin = temp['Cabin'].str[0]
    df.loc[filtro,"NewCabin"] = NewCabin

795 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
df["NewCabin"] = df["Cabin"].str[0]

509 µs ± 21.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [39]:
%%timeit
df["NewCabin"] = df["Cabin"].apply(lambda x: x.str[0])

AttributeError: 'float' object has no attribute 'str'

In [25]:
t1 = data.groupby('Ticket', dropna=False).size().sort_values(ascending=False).reset_index().rename(columns={0: "#items"})
filter = data["Cabin"].isnull()==False
t2 = data[filter].groupby(['Ticket','Cabin'], dropna=False).size().reset_index().rename(columns={0: "#items by Ticket and Cabin"})
t1.shape, t2.shape

((681, 2), (161, 3))

In [28]:
t3 = t1.merge(t2, how="left", on="Ticket")
t3 = t3.sort_values(["#items","#items by Ticket and Cabin"], ascending=[False,False])
t3

,Ticket,#items,Cabin,#items by Ticket and Cabin
0,1601,7,NaN,NaN
1,CA. 2343,7,NaN,NaN
2,347082,7,NaN,NaN
3,CA 2144,6,NaN,NaN
4,347088,6,NaN,NaN
...,...,...,...,...
695,345572,1,NaN,NaN
696,345364,1,NaN,NaN
697,343276,1,NaN,NaN
698,343275,1,NaN,NaN


In [29]:
data[df['Ticket']=='1601']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
74,75,1,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,NaN,S
169,170,0,3,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,NaN,S
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,NaN,S
643,644,1,3,"Foo, Mr. Choong",male,NaN,0,0,1601,56.4958,NaN,S
692,693,1,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,NaN,S
826,827,0,3,"Lam, Mr. Len",male,NaN,0,0,1601,56.4958,NaN,S
838,839,1,3,"Chip, Mr. Chang",male,32.0,0,0,1601,56.4958,NaN,S
